# Pandas and SQL

* Select rows
* Merge and Join
* Filter and Where
* Sort and Order
* Group
* decile
* window function qualify rank

In [45]:
import seaborn as sns
import numpy as np
import pandas as pd

df_demo = sns.load_dataset('titanic')
tips = sns.load_dataset('tips')

## Select

* select * from

In [3]:
# SQL 
# select * from table limit 10

df_demo.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [4]:
# SQL
# select sex, age from df_demo

df_demo[['sex','age']]

,sex,age
0,male,22.0
1,female,38.0
2,female,26.0
3,female,35.0
4,male,35.0
...,...,...
886,male,27.0
887,female,19.0
888,female,NaN
889,male,26.0


In [47]:
# select *, tip/total_bill as tip_rate from tips limit 5;

# With pandas, you can use the DataFrame.assign() method of a DataFrame to append a new column:
tips.assign(tip_rate=tips["tip"] / tips["total_bill"]).head(5)

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [7]:
# SQL
# select case when age < 60 then '<60' 
# when age >= 60 then '>=60'
# else 'Unknown'
# end as age_band 
# from df_demo

conditions = [df_demo['age']<60
              ,df_demo['age']>=60]
label= ['<60', '>=60']

df_demo['age_band'] = np.select(conditions,label,default='Unknown')
df_demo.age_band.value_counts()

<60        688
Unknown    177
>=60        26
Name: age_band, dtype: int64

In [8]:
df_demo['age_band'] = np.select(
    [
        df_demo['age'].between(0, 60, inclusive=False), 
        df_demo['age'].between(60, 100, inclusive=True)
    ], 
    [
        'age <60', 
        'age >=60'
    ], 
    default='Unknown'
)
df_demo.age_band.value_counts()

age <60     688
Unknown     177
age >=60     26
Name: age_band, dtype: int64

In [5]:
# SQL
# select distinct sex from df_demo
df_demo['sex'].drop_duplicates()

0      male
1    female
Name: sex, dtype: object

In [6]:
# SQL
# select sex as gender, age as age_new from df_demo
(
df_demo[['sex','age']].
 rename(columns={'sex':'gender',
                'age':'age_new'})
)

,gender,age_new
0,male,22.0
1,female,38.0
2,female,26.0
3,female,35.0
4,male,35.0
...,...,...
886,male,27.0
887,female,19.0
888,female,NaN
889,male,26.0


## Where

In [48]:
where = tips["time"]=="Dinner"
tips[where].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [82]:
tips.query("time == 'Dinner'").head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [49]:
is_dinner = tips["time"]=="Dinner"
is_dinner.value_counts()

True     176
False     68
Name: time, dtype: int64

In [50]:
# SELECT *
# FROM tips
# WHERE time = 'Dinner' AND tip > 5.00;

where =(tips["time"] == "Dinner") & (tips["tip"] > 5.00) 

tips[where].head()

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4


In [85]:
tips.query("time == 'Dinner' & tip > 5").head()

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4


In [51]:
# SELECT *
# FROM tips
# WHERE size >= 5 OR total_bill > 45;
where = (tips["size"] >= 5) | (tips["total_bill"] > 45)

tips[where].head(5)


,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6


In [86]:
tips.query("size >=5 | total_bill >45").head()

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6


In [53]:
# select *
# from demo
# where age is null;

where = df_demo["age"].isna()
df_demo[where].head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True


In [95]:
df_demo.query("age != age").head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,first_rank_fare_bysex
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,28.0
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True,237.0
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,1.0
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,3.0
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,21.0


In [97]:
df_demo.query("age.isnull()",engine="python")

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,first_rank_fare_bysex
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,28.0
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True,237.0
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,1.0
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,3.0
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True,25.0
863,0,3,female,NaN,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False,173.0
868,0,3,male,NaN,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True,210.0
878,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,122.0


In [54]:
# select *
# from demo
# where age is not null;

where = df_demo["age"].notna()
df_demo[where].head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [94]:
na = np.nan
df_demo.query("age != @na").head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,first_rank_fare_bysex
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,38.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,43.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,16.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,162.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,137.0


In [11]:
# select * from df_demo where age=60

df_demo[df_demo['age']==60]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
366,1,1,female,60.0,1,0,75.25,C,First,woman,False,D,Cherbourg,yes,False,age >=60
587,1,1,male,60.0,1,1,79.20,C,First,man,True,B,Cherbourg,yes,False,age >=60
684,0,2,male,60.0,1,1,39.00,S,Second,man,True,NaN,Southampton,no,False,age >=60
694,0,1,male,60.0,0,0,26.55,S,First,man,True,NaN,Southampton,no,True,age >=60


In [12]:
# select * from df where age=60 and sex=male

df_demo[(df_demo['age']==60)&(df_demo['sex']=='male')]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
587,1,1,male,60.0,1,1,79.20,C,First,man,True,B,Cherbourg,yes,False,age >=60
684,0,2,male,60.0,1,1,39.00,S,Second,man,True,NaN,Southampton,no,False,age >=60
694,0,1,male,60.0,0,0,26.55,S,First,man,True,NaN,Southampton,no,True,age >=60


## Order by

In [13]:
# select * from df order by age desc
df_demo.sort_values('age',ascending=False)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
630,1,1,male,80.0,0,0,30.0000,S,First,man,True,A,Southampton,yes,True,age >=60
851,0,3,male,74.0,0,0,7.7750,S,Third,man,True,NaN,Southampton,no,True,age >=60
493,0,1,male,71.0,0,0,49.5042,C,First,man,True,NaN,Cherbourg,no,True,age >=60
96,0,1,male,71.0,0,0,34.6542,C,First,man,True,A,Cherbourg,no,True,age >=60
116,0,3,male,70.5,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,no,True,age >=60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True,Unknown
863,0,3,female,NaN,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False,Unknown
868,0,3,male,NaN,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True,Unknown
878,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,Unknown


## Group by

In [55]:
# select sex,count(*) from tips group by 1;

tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

In [56]:
# count() applies the function to each column and returning the number of not null records within each
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,157,157,157,157,157,157
Female,87,87,87,87,87,87


In [57]:
# select day,avg(tip),count(*) from tips group by day;
tips.groupby("day").agg({"tip":np.mean, "day":np.size})

,tip,day
day,,
Thur,2.771452,62
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76


In [58]:
# select smoker, day, count(*), avg(tip) from tips group by 1,2;
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

tip          
             size      mean
smoker day                 
Yes    Thur  17.0  3.030000
       Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
No     Thur  45.0  2.673778
       Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895

In [14]:
# select sex,count(distinct age) from df group by sex

df_demo.groupby('sex')['age'].nunique()

sex
female    63
male      82
Name: age, dtype: int64

In [15]:
# select survived, avg(fare) from df group by survived

df_demo.groupby('survived')['fare'].mean()

survived
0    22.117887
1    48.395408
Name: fare, dtype: float64

In [18]:

# RN = ROW_NUMBER() OVER (PARTITION BY sex ORDER BY pclass ASC, embarked DESC)

df_demo['RN'] = df_demo.sort_values(['pclass','embarked'], ascending=[True,False]) \
             .groupby(['sex']) \
             .cumcount() + 1

In [19]:
df_demo.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band,RN
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,age <60,231
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,age <60,50
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,age <60,171
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,age <60,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,age <60,232


## Merge and Join

In [63]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

In [64]:
df1

,key,value
0,A,-0.971020
1,B,-0.150714
2,C,-0.018283
3,D,0.332789


In [65]:
df2

,key,value
0,B,-1.434781
1,D,-1.104017
2,D,0.723353
3,E,0.802552


In [66]:
pd.merge(df1,df2,on="key")

,key,value_x,value_y
0,B,-0.150714,-1.434781
1,D,0.332789,-1.104017
2,D,0.332789,0.723353


In [67]:
pd.merge(df1,df2,on="key",how="left")

,key,value_x,value_y
0,A,-0.971020,NaN
1,B,-0.150714,-1.434781
2,C,-0.018283,NaN
3,D,0.332789,-1.104017
4,D,0.332789,0.723353


In [68]:
pd.merge(df1,df2,on="key",how="outer")

,key,value_x,value_y
0,A,-0.971020,NaN
1,B,-0.150714,-1.434781
2,C,-0.018283,NaN
3,D,0.332789,-1.104017
4,D,0.332789,0.723353
5,E,NaN,0.802552


## UNION

In [72]:
pd.concat([df1,df2]).drop_duplicates()

,key,value
0,A,-0.971020
1,B,-0.150714
2,C,-0.018283
3,D,0.332789
0,B,-1.434781
1,D,-1.104017
2,D,0.723353
3,E,0.802552


## Aggregation and analytics functions

In [74]:
# top n rows with offset
# select * from tips order by tip desc limit 10 offset 5;  

tips.nlargest(10 +5, columns="tip").tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


In [76]:
# top nrow per group

# select * from (
#     select t.*,
#     row_number() over (partition by day order by total_bill desc) as rn 
#     from tips t
#     )
#     where rn <3 
#     order by day,rn;

(
tips.assign(
    rn = tips.sort_values(["total_bill"],ascending =False)
    .groupby(["day"])
    .cumcount() +1
    )
    .query("rn<3")
    .sort_values(["day","rn"])

)
    

,total_bill,tip,sex,smoker,day,time,size,rn
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2


In [79]:
(
    tips.assign(
        rnk = tips.groupby(["day"])["total_bill"].rank(
            method = "first",ascending = False
        )
    )
    .query("rnk<3")
    .sort_values(["day","rnk"])
)

,total_bill,tip,sex,smoker,day,time,size,rnk
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1.0
142,41.19,5.00,Male,No,Thur,Lunch,5,2.0
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1.0
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2.0
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1.0
212,48.33,9.00,Male,No,Sat,Dinner,4,2.0
156,48.17,5.00,Male,No,Sun,Dinner,6,1.0
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2.0


In [80]:
# SELECT * FROM (
#   SELECT
#     t.*,
#     RANK() OVER(PARTITION BY sex ORDER BY tip) AS rnk
#   FROM tips t
#   WHERE tip < 2
# )
# WHERE rnk < 3
# ORDER BY sex, rnk;

(
    tips[tips["tip"]<2]
    .assign(rnk_min = tips.groupby(["sex"])["tip"].rank(method="min"))
    .query("rnk_min<3")
    .sort_values(["sex","rnk_min"])

)



,total_bill,tip,sex,smoker,day,time,size,rnk_min
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.0
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.0
67,3.07,1.00,Female,Yes,Sat,Dinner,1,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,1.0


## Groupby.Rank

SQL, rank over partition by x,y order by x,y

ROW_NUMBER() OVER(PARTITION BY x ORDER BY y DESC) AS rn

In [77]:
df_demo['first_rank_fare_bysex'] = \
df_demo.sort_values(['sex','embarked','fare'], ascending=[True,True,False]) \
             .groupby(['sex','embarked'])['fare'] \
             .rank(method='first') # Row_number()

In [78]:
df_demo.sort_values(['sex','embarked','first_rank_fare_bysex']).head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,first_rank_fare_bysex
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,1.0
875,1,3,female,15.0,0,0,7.2250,C,Third,child,False,NaN,Cherbourg,yes,True,2.0
367,1,3,female,NaN,0,0,7.2292,C,Third,woman,False,NaN,Cherbourg,yes,True,3.0
780,1,3,female,13.0,0,0,7.2292,C,Third,child,False,NaN,Cherbourg,yes,True,4.0
39,1,3,female,14.0,1,0,11.2417,C,Third,child,False,NaN,Cherbourg,yes,False,5.0


## Percentile

In [40]:
df_demo['fare_percent_rank'] = df_demo['fare'].rank(pct=True) # ntile

In [41]:
df_demo['fare_percentile_rank'] =(df_demo['fare_percent_rank']*100).round()


In [42]:
df_demo.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band,RN,fare_percent_rank,fare_percentile_rank,first_rank_fare_bysex
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,age <60,231,0.086420,9.0,38.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,age <60,50,0.885522,89.0,43.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,age <60,171,0.260943,26.0,16.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,age <60,1,0.839506,84.0,162.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,age <60,232,0.296296,30.0,137.0


In [32]:
pd.value_counts(df_demo.fare_percentile_rank)


30.0     43
46.0     42
23.0     38
14.0     34
65.0     31
         ..
32.0      4
12.0      3
100.0     3
27.0      1
21.0      1
Name: fare_percentile_rank, Length: 84, dtype: int64

In [43]:
# To Do: add qcut example to calculate percentile 

## Pivot

## String functions

In [59]:
# split string into list, default separator is any whitespace
'apple-banana-orange'.split('-')


['apple', 'banana', 'orange']

In [60]:
# find the first occurrence of the specified value

'apple'.find('p')

1